In [1]:
from pypdf import PdfReader
from pypdf.errors import PdfReadError
from transformers import pipeline
import torch
import sentencepiece

In [3]:
ArticleToSummarize = "pdf_database\pdf1.pdf"

def pdfTE(pdfFile): # Function for text extraction from PDF.
    with open(pdfFile, "rb") as file: # Read in binary to handle breakline statements better (\n)
        pdfReader = PdfReader(file)
        for page in pdfReader.pages:
            yield page.extract_text() # Use of generator as keeping the whole article in the memory results in memory error.
            
def articleC(pdfFile): # Function to return the article in one string
    textCombiner = pdfTE(pdfFile) 
    textCombined = ""
    for text in textCombiner: # Loop over generator object to sum text of pages into one string
        textCombined += text
    return textCombined

def sumArticle1(pdfFile): # Function to summarize article per page
    if pdfFile.endswith(".pdf"): # First check if file extension is ".pdf" format.
        try:
            summarizer = pipeline("summarization", model="pszemraj/led-large-book-summary") # Model used from the huggingface hub (https://huggingface.co/pszemraj/led-large-book-summary)
            for text in pdfTE(pdfFile): # Iterate over generator
                summarizedPage = summarizer(text,max_length=100, min_length=50, do_sample=True)
                print(summarizedPage, end="\n")
                print()
        except PdfReadError: # Second check if file extension is ".pdf" format when first check fails.
            return 0
        except OSError: # Exception handling of OS errors, as PdfReadError doesn't catch all file extension errors.
            return 0
    else:
        return 0

def sumArticle2(pdfFile): # Function to summarize article as a whole
    if pdfFile.endswith(".pdf"): # First check if file extension is ".pdf" format.
        try:
            summarizer = pipeline("summarization", model="pszemraj/led-large-book-summary") # Model used from the huggingface hub (https://huggingface.co/pszemraj/led-large-book-summary)
            articleCombined = articleC(pdfFile) # Iterate over generator
            summarizedPage = summarizer(articleCombined,max_length=1000, min_length=200, do_sample=True)
            return summarizedPage[0]["summary_text"]
        except PdfReadError: # Second check if file extension is ".pdf" format when first check fails.
            return 0
        except OSError: # Exception handling of OS errors, as PdfReadError doesn't catch all file extension errors.
            return 0
    else:
        return 0

pdfText = articleC("pdf_database\pdf1.pdf")

text = sumArticle2(ArticleToSummarize)
print(text)

# =============================================================================
# These tests were done with function: sumArticle1
#
# Test 1 results (max length=300, min length=100):
#     - Program took 20 minutes to produce the article page for page
#     - Summaries were accurate with important and clear information
#     - Spelling errors? yes
# 
# Test 2 results (max length=10, min length = 3):
#     - Program took 2 minutes to produce the article page for page
#     - Summaries were totally not accurate, more like broken sentences
#     - Spelling errors? yes
# 
# Test 3 results (max length=50, min length = 30):
#     - Program took 4 minutes to produce the article page for page
#     - Summaries seemed somewhat accurate, sentences started the same as with test 2? weird.. is model not random?
#     - Spelling errors? yes
# 
# Test 4 results (max length=100, min length=50):
#     - Program took 5 minutes to produce the article page for page
#     - Summaries seemed accurate despite sentences being cut off
#     - Spelling errors? little
#
# These tests were done with function sumArticle2
#
# Test 1 results (max length=300, min length=100):
#     - Program took 2 minutes to produce the article
#     - Summary explains content of the pdf briefly, mentioning some techniques
#       used in the article, but not much in depth information.
#     - Seemed written as a whole, no sentences cut off.
#
# Test 2 results (max length=300, min length=100):
#     - Program took 2,5 minutes to produce the article
#     - Summary explains content of the pdf well, goes indepth quite a bit.
#     - Some sentences were cut off.
#     - Running the model with same parameters resulted in 2 different summaries.
# =============================================================================

# [{'summary_text': 'This paper discusses several different algorithmic models to predict software development time and other management parameters. The most commonly used is the Putnam model, but it is not reliable in all cases. In this paper, the authors propose a new model that predicts much more accurately than the putnam model. The authors examine several different statistical methods to predict real-time performance of this model. They also discuss several different types of prediction such as the "routine" or "learning curve," which are used to predict future performance of a particular task. They conclude that this model can be useful for predicting future performance on a broad range of industrial applications as well as specific ones.'}]

This paper discusses several different types of models used to predict software development time and cost. The most commonly used model is the Putnam SLIM model, which is based on a slope-learning curve. However, the authors believe that it is not reliable at predicting very early stages of a project. They use a different approach called the "Putnam model" to predict future performance. In this paper, they suggest a new type of model that predicts future performance much more accurately than the previous Putnam model. This new model can be implemented in a variety of ways, including: correlating statistics from real-time experiments with machine learning, using statistical methods such as rinterpreter and correlation analysis, modeling by direct observation, and using advanced statistical techniques such as weighted average. The authors also discuss several different approaches for predicting future performance of a particular kind of product or function. For example, if a company were

In [4]:
translation = pipeline("translation", model="Helsinki-NLP/opus-mt-en-nl")
translated_text = translation(text)[0]['translation_text']
print(translated_text)

Het meest gebruikte model is het Putnam SLIM model, dat gebaseerd is op een hellings-learning curve. Echter, de auteurs geloven dat het niet betrouwbaar is in het voorspellen van zeer vroege stadia van een project. Ze gebruiken een andere aanpak genaamd het "Putnam model" om toekomstige prestaties te voorspellen. In dit document suggereren ze een nieuw type model dat toekomstige prestaties veel nauwkeuriger voorspelt dan het vorige Putnam model. Dit nieuwe model kan worden toegepast op verschillende manieren, waaronder: het correleren van statistieken van real-time experimenten met machine learning, gebruik makend van statistische methoden zoals rinterpreter en correlatieanalyse, modelleren door middel van directe observatie, en gebruik makend van geavanceerde statistische technieken zoals gewogen gemiddelde. De auteurs bespreken ook verschillende benaderingen voor het voorspellen van toekomstige prestaties van een bepaald soort product of functie. Bijvoorbeeld, als een bedrijf een nie

In [ ]:
import evaluate

metric = evaluate.load("sacrebleu") # Model generated translation evaluated with DeepL generated translation

evaluation_text = "In dit artikel worden verschillende soorten modellen besproken die worden gebruikt om de ontwikkelingstijd en -kosten van software te voorspellen. Het meest gebruikte model is het Putnam SLIM model, dat gebaseerd is op een hellings-leercurve. De auteurs zijn echter van mening dat dit model niet betrouwbaar is bij het voorspellen van zeer vroege stadia van een project. Ze gebruiken een andere benadering, het 'Putnam-model', om toekomstige prestaties te voorspellen. In dit artikel stellen ze een nieuw type model voor dat toekomstige prestaties veel nauwkeuriger voorspelt dan het vorige Putnam-model. Dit nieuwe model kan op verschillende manieren worden geïmplementeerd, waaronder: het correleren van statistieken van real-time experimenten met machine learning, het gebruik van statistische methoden zoals rinterpreter en correlatieanalyse, modellering door directe observatie en het gebruik van geavanceerde statistische technieken zoals gewogen gemiddelden. De auteurs bespreken ook verschillende benaderingen voor het voorspellen van toekomstige prestaties van een bepaald soort product of functie. Als een bedrijf bijvoorbeeld een nieuw stuk machine zou introduceren, zou het in staat moeten zijn om het snel en op een hoog niveau te produceren over een lange periode."

translated_text = "Het meest gebruikte model is het Putnam SLIM model, dat gebaseerd is op een hellings-learning curve. Echter, de auteurs geloven dat het niet betrouwbaar is in het voorspellen van zeer vroege stadia van een project. Ze gebruiken een andere aanpak genaamd het 'Putnam model' om toekomstige prestaties te voorspellen. In dit document suggereren ze een nieuw type model dat toekomstige prestaties veel nauwkeuriger voorspelt dan het vorige Putnam model. Dit nieuwe model kan worden toegepast op verschillende manieren, waaronder: het correleren van statistieken van real-time experimenten met machine learning, gebruik makend van statistische methoden zoals rinterpreter en correlatieanalyse, modelleren door middel van directe observatie, en gebruik makend van geavanceerde statistische technieken zoals gewogen gemiddelde. De auteurs bespreken ook verschillende benaderingen voor het voorspellen van toekomstige prestaties van een bepaald soort product of functie. Bijvoorbeeld, als een bedrijf een nieuw stuk machines zou moeten kunnen introduceren, zou het snel en op een hoog niveau van tijd moeten produceren."

predictions = [
    translated_text
]
references = [
    [
        evaluation_text
    ]
]
metric.compute(predictions=predictions, references=references)

### ==============================
#{'score': 53.86425158362457,
#'counts': [149, 112, 95, 80],
#'totals': [173, 172, 171, 170],
#'precisions': [86.1271676300578,
# 65.11627906976744,
# 55.55555555555556,
# 47.05882352941177],
#'bp': 0.8704644809074915,
#'sys_len': 173,
#'ref_len': 197}
### ===============================